In [1]:
# ─── CELL 0  (ENV + DATA LOAD & PRE-PROCESSING with cyclical features and data augmentation) ───
import torch, os, sys, platform, subprocess, textwrap, random
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device name :", torch.cuda.get_device_name(0))
    dummy = torch.randn(256, 256).to("cuda")
    print("Tensor is on :", dummy.device)

# dont erase the follwoing commented line:
data_dir = "/kaggle/input/unipd-deep-learning-2025-challenge-2/"

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Data & Loader parameters
torch.set_num_threads(2)
NUM_WORKERS = 4
batch_size = 128
input_len  = 90
output_len = 30

# Augmentation parameters
noise_std      = 0.01             # jitter noise
scaling_range  = (0.9, 1.1)         # magnitude scaling range
dropout_prob   = 0.1              # feature/channel dropout probability

# ---------------------------------------------------------------------------
# 1. Load raw dataset
# ---------------------------------------------------------------------------
data = pd.read_csv(data_dir + 'train_dataset.csv', index_col=[0, 1])

# ---------------------------------------------------------------------------
# 2. Add cyclical *day-of-year* features (sine & cosine)
# ---------------------------------------------------------------------------
idx_level_1 = data.index.get_level_values(1)

if np.issubdtype(idx_level_1.dtype, np.integer):
    day_number = idx_level_1.to_numpy()
else:
    parsed = pd.to_datetime(idx_level_1, errors="coerce")
    if parsed.notna().all():
        day_number = parsed.dayofyear.to_numpy()
    else:
        day_number = np.arange(len(idx_level_1)) % 365

period = 365.0

data["doy_sin"] = np.sin(2 * np.pi * day_number / period)
data["doy_cos"] = np.cos(2 * np.pi * day_number / period)

# ---------------------------------------------------------------------------
# 3. Convert to 3-D array  (station, day, feature)
# ---------------------------------------------------------------------------
stations = [station.values for _, station in data.groupby(level=0)]
data_arr = np.stack(stations, axis=0)

n_stations, n_days, n_features = data_arr.shape
print(f"Number of weather stations: {n_stations}")
print(f"Number of days of data   : {n_days}")
print(f"Number of weather variables (incl. cyclical): {n_features}")

# ---------------------------------------------------------------------------
# 4. Normalisation (mean / std computed on training period only)
# ---------------------------------------------------------------------------
feature_cols      = data.columns.tolist()  # var1 … var76 + doy_sin, doy_cos
val_start_timestep = n_days - 30           # last 30 timesteps = validation target

mask_train_period = data.index.get_level_values(1) < val_start_timestep
train_means       = data.loc[mask_train_period, feature_cols].mean()
train_stds        = data.loc[mask_train_period, feature_cols].std()

data_norm = data.copy()
data_norm[feature_cols] = (data_norm[feature_cols] - train_means) / train_stds

stations_norm  = [station.values for _, station in data_norm.groupby(level=0)]
data_arr_norm  = np.stack(stations_norm, axis=0)

print("\n─ Normalisation summary ─")
print("Normalised array shape:", data_arr_norm.shape)   # (stations, days, features)

# ---------------------------------------------------------------------------
# 5. Dataset & DataLoader with On-the-Fly Augmentation
# ---------------------------------------------------------------------------
val_input_start = n_days - (input_len + output_len)

class WeatherWindowDataset(Dataset):
    def __init__(self, data_3d: np.ndarray, mode="train"):
        self.data          = data_3d.astype(np.float32)
        self.mode          = mode
        self.in_len        = input_len
        self.out_len       = output_len
        self.jitter_std    = noise_std
        self.scaling_low, self.scaling_high = scaling_range
        self.dropout_prob  = dropout_prob

        n_stations, n_days, _ = self.data.shape
        if mode == "train":
            last_start = val_input_start - self.in_len - self.out_len
            idx = [(sid, t) for sid in range(n_stations) for t in range(0, last_start + 1)]
        else:
            idx = [(sid, val_input_start) for sid in range(n_stations)]
        self.indices = np.asarray(idx, dtype=np.int32)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        sid, start = self.indices[idx]
        window = self.data[sid, start : start + self.in_len + self.out_len]
        x = window[: self.in_len]    # (input_len, features)
        y = window[self.in_len :]    # (output_len, features)

        if self.mode == "train":
            # 1) Jitter noise
            if self.jitter_std > 0:
                x = x + self.jitter_std * np.random.randn(*x.shape).astype(np.float32)
            # 2) Magnitude scaling
            scales = np.random.uniform(self.scaling_low,
                                       self.scaling_high,
                                       size=(1, x.shape[1])).astype(np.float32)
            x = x * scales
            # 3) Channel dropout
            mask = np.random.binomial(1, 1 - self.dropout_prob,
                                      size=(1, x.shape[1])).astype(np.float32)
            x = x * mask

        return torch.from_numpy(x), torch.from_numpy(y)

# Instantiate datasets
train_ds = WeatherWindowDataset(data_arr_norm, mode="train")
val_ds   = WeatherWindowDataset(data_arr_norm, mode="val")

print(f"Train samples : {len(train_ds):,}")
print(f"Val   samples : {len(val_ds):,}")

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=batch_size,
                          shuffle=True,  pin_memory=True, num_workers=NUM_WORKERS)
val_loader   = DataLoader(val_ds,   batch_size=batch_size,
                          shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)

print(f"Train loader: {len(train_loader)} batches of {batch_size}")
print(f"Val   loader: {len(val_loader)} batches of {batch_size}")



PyTorch: 2.6.0+cu124
CUDA available: True
Device name : Tesla T4
Tensor is on : cuda:0
Number of weather stations: 422
Number of days of data   : 695
Number of weather variables (incl. cyclical): 78

─ Normalisation summary ─
Normalised array shape: (422, 695, 78)
Train samples : 192,432
Val   samples : 422
Train loader: 1504 batches of 128
Val   loader: 4 batches of 128


In [2]:
import math, random, copy
import numpy as np
import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR

# --------------------- Device Setup ---------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --------------------- Hyperparameters ---------------------
SEED_GRU_NEW1 = 23   # first substituted GRU seed
SEED_GRU_NEW2 = 105  # second substituted GRU seed
SEED_GRU_NEW3 = 100  # third substituted GRU seed
SEED_GRU1    = 104  # original GRU seed
SEED_GRU2    = 22   # second GRU seed
NUM_EPOCHS   = 50
PATIENCE     = 10
BASE_LR      = 3e-4
WEIGHT_DECAY = 1e-5
CLIP_NORM    = 1.0
TF_EPOCHS    = 10

# GRU hyperparams (used for all GRUs)
gru_hidden, gru_layers, gru_dropout = 256, 2, 0.01

input_size = n_features
input_len  = input_len
output_len = output_len

# ------------------ Model Definitions ------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class TransformerForecast(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dim_feedforward, dropout, out_len, max_len):
        super().__init__()
        self.input_proj = nn.Linear(input_size, d_model)
        self.pos_enc    = PositionalEncoding(d_model, max_len)
        layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.encoder    = nn.TransformerEncoder(layer, num_layers)
        self.fc_out     = nn.Linear(d_model, input_size * out_len)
        self.out_len    = out_len

    def forward(self, x, y=None, tf_ratio=None):
        x = self.pos_enc(self.input_proj(x))
        enc = self.encoder(x)
        h   = enc[:, -1, :]
        return self.fc_out(h).view(x.size(0), self.out_len, -1)

class GRUSeq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, out_len):
        super().__init__()
        self.out_len = out_len
        self.encoder = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.decoder = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc_out  = nn.Linear(hidden_size, input_size)

    def forward(self, x, y=None, tf_ratio=1.0):
        _, h = self.encoder(x)
        dec_in = x[:, -1:, :]
        outs = []
        for t in range(self.out_len):
            out, h = self.decoder(dec_in, h)
            pred = self.fc_out(out.squeeze(1))
            outs.append(pred.unsqueeze(1))
            take_gt = (y is not None) and (torch.rand(1).item() < tf_ratio)
            dec_in  = y[:, t:t+1, :] if take_gt else pred.unsqueeze(1)
        return torch.cat(outs, 1)

# ---------------- Helper: Teacher-forcing schedule ----------------
def tf_ratio(epoch: int) -> float:
    return max(0.0, 1.0 - (epoch - 1)/(TF_EPOCHS - 1))

# ---------------- Training Helper ----------------
def train_model(model, train_loader, val_loader, num_epochs):
    model = model.to(device)
    scaler = GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
    scheduler = OneCycleLR(optimizer, max_lr=BASE_LR, epochs=num_epochs,
                           steps_per_epoch=len(train_loader), pct_start=0.4)
    best_val, best_state, no_imp = float('inf'), None, 0
    mae_loss = nn.L1Loss()
    for ep in range(1, num_epochs + 1):
        model.train()
        mse_acc = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            with autocast(device_type="cuda"):
                preds = model(xb, yb, tf_ratio(ep))
                loss  = mae_loss(preds, yb)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            mse_acc += torch.mean((preds - yb) ** 2).item() * xb.size(0)
        train_mse = mse_acc / len(train_loader.dataset)
        model.eval()
        mae_sum = 0.0
        with torch.no_grad(), autocast(device_type="cuda"):
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                yhat = model(xb, None, tf_ratio(1))
                mae_sum += torch.abs(yhat - yb).sum().item()
        val_mae = mae_sum / len(val_loader.dataset)
        print(f"Epoch {ep}/{num_epochs} | LR {scheduler.get_last_lr()[0]:.1e} | TF {tf_ratio(ep):.2f}"  \
              f" | Train MSE {train_mse:.4f} | Val MAE {val_mae:.2f}")
        if val_mae < best_val:
            best_val, best_state, no_imp = val_mae, copy.deepcopy(model.state_dict()), 0
        else:
            no_imp += 1
            if no_imp >= PATIENCE:
                print("Early stopping.")
                break
    model.load_state_dict(best_state)
    return model, best_val

# ─── Train GRUSeq2Seq (new1, seed=23) ───
random.seed(SEED_GRU_NEW1)
np.random.seed(SEED_GRU_NEW1)
torch.manual_seed(SEED_GRU_NEW1)
torch.cuda.manual_seed_all(SEED_GRU_NEW1)
print(f"=== TRAINING GRUSeq2Seq (new1, seed={SEED_GRU_NEW1}) ===")
model_gru3, best_val_gru3 = train_model(GRUSeq2Seq(input_size, gru_hidden, gru_layers, gru_dropout, output_len), train_loader, val_loader, NUM_EPOCHS)

# ─── Train GRUSeq2Seq (new2, seed=105) ───
random.seed(SEED_GRU_NEW2)
np.random.seed(SEED_GRU_NEW2)
torch.manual_seed(SEED_GRU_NEW2)
torch.cuda.manual_seed_all(SEED_GRU_NEW2)
print(f"=== TRAINING GRUSeq2Seq (new2, seed={SEED_GRU_NEW2}) ===")
model_gru4, best_val_gru4 = train_model(GRUSeq2Seq(input_size, gru_hidden, gru_layers, gru_dropout, output_len), train_loader, val_loader, NUM_EPOCHS)

# ─── Train GRUSeq2Seq (new3, seed=100) ───
random.seed(SEED_GRU_NEW3)
np.random.seed(SEED_GRU_NEW3)
torch.manual_seed(SEED_GRU_NEW3)
torch.cuda.manual_seed_all(SEED_GRU_NEW3)
print(f"=== TRAINING GRUSeq2Seq (new3, seed={SEED_GRU_NEW3}) ===")
model_gru5, best_val_gru5 = train_model(GRUSeq2Seq(input_size, gru_hidden, gru_layers, gru_dropout, output_len), train_loader, val_loader, NUM_EPOCHS)

# ─── Train GRUSeq2Seq (orig, seed=104) ───
random.seed(SEED_GRU1)
np.random.seed(SEED_GRU1)
torch.manual_seed(SEED_GRU1)
torch.cuda.manual_seed_all(SEED_GRU1)
print(f"=== TRAINING GRUSeq2Seq (orig, seed={SEED_GRU1}) ===")
model_gru1, best_val_gru1 = train_model(GRUSeq2Seq(input_size, gru_hidden, gru_layers, gru_dropout, output_len),train_loader, val_loader, NUM_EPOCHS)

# ─── Train GRUSeq2Seq (rep, seed=22) ───
random.seed(SEED_GRU2)
np.random.seed(SEED_GRU2)
torch.manual_seed(SEED_GRU2)
torch.cuda.manual_seed_all(SEED_GRU2)
print(f"=== TRAINING GRUSeq2Seq (rep, seed={SEED_GRU2}) ===")
model_gru2, best_val_gru2 = train_model(GRUSeq2Seq(input_size, gru_hidden, gru_layers, gru_dropout, output_len),train_loader, val_loader, NUM_EPOCHS)

# Final summary
print(
    f"Best Val MAE → GRU(new1): {best_val_gru3:.2f}, GRU(new2): {best_val_gru4:.2f}, GRU(new3): {best_val_gru5:.2f}, "
    f"GRU(orig): {best_val_gru1:.2f}, GRU(rep): {best_val_gru2:.2f}"
)


Using device: cuda
=== TRAINING GRUSeq2Seq (new1, seed=23) ===
Epoch 1/50 | LR 1.4e-05 | TF 1.00 | Train MSE 0.7661 | Val MAE 1658.96
Epoch 2/50 | LR 1.9e-05 | TF 0.89 | Train MSE 0.5566 | Val MAE 1559.68
Epoch 3/50 | LR 2.8e-05 | TF 0.78 | Train MSE 0.4959 | Val MAE 1495.30
Epoch 4/50 | LR 4.0e-05 | TF 0.67 | Train MSE 0.4941 | Val MAE 1441.91
Epoch 5/50 | LR 5.4e-05 | TF 0.56 | Train MSE 0.5147 | Val MAE 1352.63
Epoch 6/50 | LR 7.1e-05 | TF 0.44 | Train MSE 0.5421 | Val MAE 1429.25
Epoch 7/50 | LR 9.1e-05 | TF 0.33 | Train MSE 0.5817 | Val MAE 1425.59
Epoch 8/50 | LR 1.1e-04 | TF 0.22 | Train MSE 0.6354 | Val MAE 1459.25
Epoch 9/50 | LR 1.3e-04 | TF 0.11 | Train MSE 0.7037 | Val MAE 1458.87
Epoch 10/50 | LR 1.6e-04 | TF 0.00 | Train MSE 0.7805 | Val MAE 1462.21
Epoch 11/50 | LR 1.8e-04 | TF 0.00 | Train MSE 0.7532 | Val MAE 1461.95
Epoch 12/50 | LR 2.0e-04 | TF 0.00 | Train MSE 0.7205 | Val MAE 1414.78
Epoch 13/50 | LR 2.2e-04 | TF 0.00 | Train MSE 0.6851 | Val MAE 1444.96
Epoch 14/5

In [3]:
# ─── CELL 2: Inference & Submission (Updated Models) ───
import torch
import numpy as np
import pandas as pd

# Ensemble inference over five models
models = [model_gru3, model_gru4, model_gru5, model_gru1, model_gru2]
for m in models:
    m.eval()

# 1) Prepare input window
input_window = data_arr_norm[:, -input_len:, :].astype(np.float32)  # (stations, input_len, features)
x_in = torch.from_numpy(input_window).to(device)

# 2) Generate predictions
preds = []
with torch.no_grad():
    preds.append(model_gru3(x_in, None, tf_ratio(0)))
    preds.append(model_gru4(x_in, None, tf_ratio(0)))
    preds.append(model_gru5(x_in, None, tf_ratio(0)))
    preds.append(model_gru1(x_in, None, tf_ratio(0)))
    preds.append(model_gru2(x_in, None, tf_ratio(0)))
# move to numpy
preds = [p.cpu().numpy() for p in preds]

# 3) Compute ensemble weights from validation MAEs
val_scores = np.array([best_val_gru3, best_val_gru4, best_val_gru5, best_val_gru1, best_val_gru2])
weights = 1.0 / val_scores
weights /= weights.sum()

# 4) Weighted ensemble
ensemble_norm = np.tensordot(weights, np.stack(preds, axis=0), axes=(0,0))  # (stations, out_len, features)

# 5) Denormalize
means = train_means.values.reshape(1,1,-1)
stds  = train_stds.values.reshape(1,1,-1)
ensemble_raw = ensemble_norm * stds + means

# 6) Discard cyclical features (last two)
ensemble_vars = ensemble_raw[:, :, :-2]  # shape (stations, out_len, var_count)

# 7) Build submission DataFrame
n_stations, H, n_vars = ensemble_vars.shape
# sanity check: ensure only 76 variables are output
assert n_vars == 76, f"Expected 76 variables, got {n_vars}"
rows = []
for sid in range(n_stations):
    for t in range(H):
        idx = f"{sid}_{t}"
        rows.append([idx] + ensemble_vars[sid, t].tolist())
var_cols = [f"var{i+1}" for i in range(n_vars)]
sub_df = pd.DataFrame(rows, columns=["id"] + var_cols)

# 8) Save
sub_df.to_csv("submission.csv", index=False)
print("Saved submission.csv with shape", sub_df.shape)


Saved submission.csv with shape (12660, 77)
